In [3]:
%load_ext tensorboard

from model import Net, train
from data import *
from utils import imshow

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
from distributed_setup import run

from tensorboard.plugins.hparams import api as hp

HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1, 1000]))
HP_NUM_WORKERS = hp.HParam('num_workers', hp.Discrete([0, 6]))
METRIC_ACCURACY = 'accuracy'
METRIC_RUNTIME = 'runtime'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_BATCH_SIZE, HP_NUM_WORKERS],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy'), hp.Metric(METRIC_RUNTIME, display_name='Run Time')],
  )

model = Net()

run(model, train, trainset, HP_BATCH_SIZE, HP_NUM_WORKERS)

2023-06-26 18:28:26.323141: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-26 18:28:26.464115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 18:28:27.459220: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Running on  4 cuda:0
Files already downloaded and verified
Files already downloaded and verified
distributed_training on: 0
Files already downloaded and verified
Files already downloaded and verified
distributed_training on: 1
Files already downloaded and verified
Files already downloaded and verified
distributed_training on: 2
Files already downloaded and verified
Files already downloaded and verified
distributed_training on: 3


ProcessRaisedException: 

-- Process 3 terminated with the following error:
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/project/studios/this_studio/image_classifier/distributed_setup.py", line 32, in distributed_training
    sampled_dataloader = prep_data(dataset, batch_size, world_size, rank, num_workers)
  File "/project/studios/this_studio/image_classifier/distributed_setup.py", line 26, in prep_data
    return DataLoader(dataset, batch_size=batch_size, num_workers=num_workers, sampler=sampler) # type: ignore
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 237, in __init__
    if num_workers < 0:
TypeError: '<' not supported between instances of 'HParam' and 'int'


In [ ]:
import torch
from model import test

PATH = './cifar_net_gpu_distributed.pth'


dataiter = iter(testloader)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

reloaded_model = Net()
reloaded_model.load_state_dict(torch.load(PATH))

outputs = reloaded_model(images)

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}' for j in range(4)))

test(reloaded_model)